# Assignment 7

# Basic imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from pagayapro.paths.data_paths import ASSIGNMENT7_DATA

# Analyzing the LC-2017-04 secondary pool

Start with loading the secondary pool's features

In [ ]:
pool_features = pd.read_parquet(os.path.join(ASSIGNMENT7_DATA, "lc_pool_features.parquet"))

Explore your data: how many loans are there in the dataset? What are their terms? At what mob are these loans purchased? Create a plot which describes the distribution of purchase mobs in the dataset; how old is the oldest loan? how young is the youngest one?

Create plots to describe the distribution of various important features in the pool (credit score, interest rate, dti, loan amount etc). 

Do these distributions seem similar to those we've seen in previous loan sets? Let's compare these distributions with historical distribution taken from the same platform. Read the following file, which contains data about loans issued by LC in 2014-2015:

In [10]:
historical_features = pd.read_parquet(os.path.join(ASSIGNMENT7_DATA, "historical_lc_features.parquet"))

Add the corresponding historical distribution to each of the plots you output above for the seconday pool. Do these distributions look similar to you? What do you think is the reason for the differences between them?

# Test set creation

## Train-test split 

Perform a train-test split to the historical data as describe in the notebook. 

In [ ]:
train_features = #write your code here
test_features = #write your code here

## Read payments files

Read the historical payments file from the folowing path

In [18]:
payments= pd.read_parquet(os.path.join(ASSIGNMENT7_DATA, "historical_lc_payments.parquet"))

Explore the payments table; display its first five rows; what's its shape? how many unique loans does it describe? What does each row of the payments table describe?

Split the payments fils according to your train-test split.

In [ ]:
train_payments = #write your code here
test_payments = #write your code here

## Turning your test set into a mock secondary pool

Our test set currently describes a set of complete loans, i.e. loans that are to be purchased on their date of origination. In order to adapt to the current pool, we want to test our model on a set of loans of varying life-spans, which are all bought at the same time.

Change your test set to be so. What is the distribution of purchase mobs in your test set after the change? Is it similar to that of the secondary pool?

Plot distributions of key features in your test set. Do they resemble the ones in the train set? How about the ones in the pool?

## Target creation 

The IRR of a secondary loan is computed differently from a loan at its origination; see the Notion page for further explanation. Use the payments file above to compute the IRR of each loan in your secondary-test set using the proper manipulation of your payments table.

# Training models 

## Predicting using an ordinary IRR predictor

Fit an IRR predicting model to your train set (this includes creating the IRR targets for these loans), and use it for predicting IRR on your test set.

Evaluate your model: create a scatterplot of predicted IRR vs true IRR for your test set and add a regression line to it. Is this line close to the identity line? Is it skewed in any dirction?

_Recall_. We have already seen that our IRR predicting model tended to perform well in expactation but not as a predictor for individual loans. Thus, a better representation of the model's performance my be to use a binned scatterplot, rather than an ordinary one.

How would you describe the model's predictions? Are they overly optimistic/conservative in your opinion?

## Predicting using a vanilla model

Generally, when training a model, we want the train set's distributions to be similar to those of the test set. However, we've already seen here that this is not the case. So, perhaps the model's predictive abilty can be improved by training on secondary loans instead of complete loans?

Perform the same processing you performed above for the test set in order to make your training set into a mock-secondary pool.

Re-fit your IRR predicting model to the modified train set and use your model to predict IRR on loans in your train set. Do not perform any additional tuning to your model just yet.

Evaluate your new model's prediction using a (binned) scatterplot as for predicted IRR vs true IRR on the test set. Compare these results to those of the previous model, which one of the models would you consider a better predictor?

Perform any additional tuning to the model you'd like and see if there is any improvement to the model's predictions.

Compare the volume-return curves of the two models; would you consider one model better than the other from a business perspective?

## Features engineering using partial performance information

So far, in transforming our data from complete loan data to secondary, we mostly dropped rows from our payments data. However, these rows contain information regarding the loans' performance in early mobs, which we know to be indicators of the loans' overall performance. AND! They are also available to us in the new secondary pool!!!

So let's use them in order to create new features for our model to train on! Review the payments table and use the data inside the table from before its purchase mob in order to create new features by your model to train on. 

_Note_. As your payments describe loans with varying life-spans, some aggregations on the payments information may require some normalization, according to the number of months until the loan's purchase.

Fit another IRR predicting model to your training set, enriched with the new features. You may perform any parameter tuning you'd like to this model.

Evaluate your new model by comparing its prediction to the true IRR values, as done for the previous models. How does the model's prections fit the identity line? How do they compare to the previous models?

Plot the new model's volume-return curve, and compare it to that of the previous models. Which of these would you recommend for deploymnet to production?

# Negotiating the pool

## Discounts

What is the optimal IRR that we can expect to obtain assuming that we are required to buy at least 50% of the pools volume (in \$)?

As mentioned in the Notion page, it is often possible to bargain with the platform on the price we pay for buying loans from a secondary pool. However, actually computing the effect of discounting a loans' principal out on the eventual IRR is hard to do analytically. One way of doing this is to retrain your model with targets adjusted for different discounts and predict using such a model.

Let's do just that- recalibrate your model to account for different possible discount on the principal; recompute your targets and train your model to predict on the recalibrated targets. What discount should you ask for in order to get an increase of 3% in your selection's IRR?

## (*) Preserving pool distributions 

Another factor which may inhibit the purchase of a secondary pool is if the issuing company (i.e. LC) is worried that our selection decreases the valus of its pool (e.g. if we select all loans in the pool with above median credit score). One cause for such concern is if the set of loans remaining in the pool have different ditributions (with respect to certain selected features) than the original pool.

Use your model to predict on the LC-2017-04 secondary pool, and choose the top 50% volume (in \$) with respect to model score. Compare distributions of `fico`, `int_rate` and `dti` between your selection, the entire pool, and the set of loans remaining in the pool after selection. Do these distributions look similar? Which do you belive represent a ''better'' population of borrowers? 

Write an algorithm that creates a selection of optimal IRR for 50%, under the constraint that the means of `fico`, `int_rate` and `dti` of the loans remaining in the pool after selection are equal (or better) than that of the original pool.

Can your algorithm also assure that the distributions of these features are unchanged?